# Langkah 1: Feature Engineering (FE)

**Tujuan:** Mengeksekusi Rencana Taktis dari EDA (Langkah 0) untuk membangun sebuah *feature matrix* (tabel fitur).

**Output:** Sebuah file `user_features.csv` di mana setiap baris adalah satu `user_id` yang *aktif*, dan setiap kolom adalah fitur yang telah kita desain (misal: `completion_rate`, `pref_type_tutorial_pct`, dll.).

## 1. Setup & Pemuatan Data

Kita muat *library* dan 4 DataFrame gabungan kita, sama seperti di EDA.

In [2]:
import pandas as pd
import numpy as np
import os

print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
DATA_DIR = '../data/'
OUTPUT_DIR = '../data/' # Kita akan simpan hasil FE kembali ke folder data

try:
    # 1. Muat & Gabungkan Users
    df_users_en = pd.read_csv(os.path.join(DATA_DIR, 'users_en.csv'))
    df_users_fr = pd.read_csv(os.path.join(DATA_DIR, 'users_fr.csv'))
    df_users = pd.concat([df_users_en, df_users_fr], ignore_index=True)

    # 2. Muat & Gabungkan Items
    df_items_en = pd.read_csv(os.path.join(DATA_DIR, 'items_en.csv'))
    df_items_fr = pd.read_csv(os.path.join(DATA_DIR, 'items_fr.csv'))
    df_items = pd.concat([df_items_en, df_items_fr], ignore_index=True)

    # 3. Muat & Gabungkan Explicit Ratings
    df_explicit_en = pd.read_csv(os.path.join(DATA_DIR, 'explicit_ratings_en.csv'))
    df_explicit_fr = pd.read_csv(os.path.join(DATA_DIR, 'explicit_ratings_fr.csv'))
    df_explicit = pd.concat([df_explicit_en, df_explicit_fr], ignore_index=True)

    # 4. Muat & Gabungkan Implicit Ratings
    df_implicit_en = pd.read_csv(os.path.join(DATA_DIR, 'implicit_ratings_en.csv'))
    df_implicit_fr = pd.read_csv(os.path.join(DATA_DIR, 'implicit_ratings_fr.csv'))
    df_implicit = pd.concat([df_implicit_en, df_implicit_fr], ignore_index=True)

    print("Data loaded and concatenated successfully:")
    print(f"Total Users: {df_users.shape}")
    print(f"Total Items: {df_items.shape}")
    print(f"Total Explicit Ratings: {df_explicit.shape}")
    print(f"Total Implicit Ratings: {df_implicit.shape}")

except FileNotFoundError as e:
    print(f"Error: {e}")

Data loaded and concatenated successfully:
Total Users: (131247, 2)
Total Items: (2618, 12)
Total Explicit Ratings: (88998, 5)
Total Implicit Ratings: (275735, 3)


## 2. Pra-pemrosesan & Pembersihan Data

Melaksanakan Rencana Taktis 2: Membersihkan `df_items` dan menyiapkan data.

In [4]:
# Membersihkan string '[]' (daftar kosong) di df_items
df_items['Software'] = df_items['Software'].replace('[]', 'No Tag')
df_items['Theme'] = df_items['Theme'].replace('[]', 'No Tag')

# Memilih hanya kolom yang kita butuhkan dari df_items untuk di-merge nanti
df_items_cleaned = df_items[['item_id', 'type', 'Software', 'Theme', 'duration']].copy()

# Membersihkan kolom 'type' (jika ada NaN)
df_items_cleaned['type'] = df_items_cleaned['type'].fillna('Unknown')

print("df_items cleaned and preprocessed.")
print(df_items_cleaned.head())

df_items cleaned and preprocessed.
   item_id      type      Software                             Theme  duration
0      510  tutorial  ['OneDrive']                      ['Discover']      42.0
1      511  tutorial  ['OneDrive']  ['Share', 'Produce', 'Organize']     122.0
2      512  tutorial  ['OneDrive']          ['Share', 'Collaborate']     176.0
3      513  tutorial  ['OneDrive']           ['Produce', 'Organize']      46.0
4      514  tutorial  ['OneDrive']           ['Produce', 'Organize']     163.0


## 3. Rencana Taktis 1: Filter Populasi User

Membuang ~75% user "turis" (yang memiliki <= 5 interaksi) dan **hanya** menyimpan ~25% user "aktif". Ini adalah langkah terpenting untuk mengurangi *noise*.

In [5]:
MIN_INTERACTIONS = 5 # Ambang batas kita

# 1. Hitung interaksi per user
user_interaction_count = df_explicit.groupby('user_id').size()

# 2. Filter untuk mendapatkan user_id yang aktif
active_user_ids = user_interaction_count[user_interaction_count > MIN_INTERACTIONS].index

print(f"Total user unik original: {df_explicit['user_id'].nunique()}")
print(f"Total user 'aktif' (>{MIN_INTERACTIONS} interaksi): {len(active_user_ids)}")
print(f"% user yang akan dianalisis: {(len(active_user_ids) / df_explicit['user_id'].nunique()) * 100:.2f}%")

# 3. Buat DataFrame fitur utama kita, di-indeks oleh user_id yang aktif
# Ini adalah tabel yang akan kita "isi" dengan fitur
user_features = pd.DataFrame(index=active_user_ids)

# 4. Filter data rating kita agar hanya berisi user aktif (ini membuat sisa proses jauh lebih cepat)
df_explicit_active = df_explicit[df_explicit['user_id'].isin(active_user_ids)]
df_implicit_active = df_implicit[df_implicit['user_id'].isin(active_user_ids)]

Total user unik original: 10611
Total user 'aktif' (>5 interaksi): 2692
% user yang akan dianalisis: 25.37%


## 4. Rencana Taktis 3: Fitur Engagement (Perilaku Tontonan)

Membuat fitur `total_..._interactions`, `completion_rate`, `drop_off_rate`, dan `avg_watch_pct`.

In [6]:
print("Engineering Engagement Features...")

# 1. Total Interactions (dari data yang sudah difilter aktif)
user_features['total_explicit_interactions'] = df_explicit_active.groupby('user_id').size()
user_features['total_implicit_interactions'] = df_implicit_active.groupby('user_id').size()

# 2. Rata-rata Watch Percentage
user_features['avg_watch_pct'] = df_explicit_active.groupby('user_id')['watch_percentage'].mean()

# 3. Completion & Drop-off Rate (Fitur Bimodal kita)
def calculate_rates(group):
    total_count = len(group)
    completion_count = (group >= 9).sum() # rating 9 atau 10
    drop_off_count = (group <= 2).sum() # rating 1 atau 2
    return pd.Series({
        'completion_rate': (completion_count / total_count) * 100,
        'drop_off_rate': (drop_off_count / total_count) * 100
    })

rate_features = df_explicit_active.groupby('user_id')['rating'].apply(calculate_rates)

# Gabungkan fitur rate ke tabel utama
user_features = user_features.join(rate_features)

print("Engagement Features Complete.")
print(user_features.head())

Engineering Engagement Features...
Engagement Features Complete.
                         total_explicit_interactions  \
user_id                                                
30      completion_rate                           15   
        drop_off_rate                             15   
72      completion_rate                            8   
        drop_off_rate                              8   
114     completion_rate                           82   

                         total_implicit_interactions  avg_watch_pct  \
user_id                                                               
30      completion_rate                          221      21.333333   
        drop_off_rate                            221      21.333333   
72      completion_rate                           22      99.125000   
        drop_off_rate                             22      99.125000   
114     completion_rate                          168      98.426829   

                             rating  
user_i

## 5. Rencana Taktis 4 & 5: Fitur Preferensi Konten & Topik

Ini adalah langkah yang paling berat. Kita perlu menggabungkan (`merge`) `df_explicit_active` dengan `df_items_cleaned` untuk menghubungkan perilaku user dengan metadata item.

In [7]:
print("Merging active ratings with item metadata...")
# Gabungkan data interaksi aktif dengan data item yang sudah bersih
merged_df = df_explicit_active.merge(df_items_cleaned, on='item_id', how='left')

print(f"Merge complete. Shape: {merged_df.shape}")
print(merged_df.head())

Merging active ratings with item metadata...
Merge complete. Shape: (73669, 9)
   user_id  item_id  watch_percentage           created_at  rating      type  \
0   277945      510               100  2018-11-20 16:21:37      10  tutorial   
1   277945      511                81  2018-12-06 17:27:29       9  tutorial   
2   277945      512               100  2018-11-20 16:23:15      10  tutorial   
3   277945      619               100  2019-05-22 14:12:35      10  tutorial   
4   277945      733               100  2019-03-18 16:47:19      10  tutorial   

                Software                             Theme  duration  
0           ['OneDrive']                      ['Discover']      42.0  
1           ['OneDrive']  ['Share', 'Produce', 'Organize']     122.0  
2           ['OneDrive']          ['Share', 'Collaborate']     176.0  
3  ['Outlook', 'Groups']                   ['Collaborate']      94.0  
4              ['Excel']          ['Discover', 'Research']      67.0  


In [8]:
print("Engineering Content Preference Features...")

# 1. Rata-rata Durasi Tontonan
user_features['avg_duration_watched'] = merged_df.groupby('user_id')['duration'].mean()

# 2. Preferensi Tipe Konten (Tutorial vs Webcast vs Use Case)
# Kita gunakan pivot_table untuk ini. Ini adalah cara yang sangat efisien.
type_prefs = merged_df.groupby('user_id')['type'].value_counts(normalize=True).unstack(fill_value=0)

# Ganti nama kolom agar lebih deskriptif
type_prefs.columns = [f'pref_type_{col.lower().replace(" ", "_")}_pct' for col in type_prefs.columns]

# Gabungkan ke tabel fitur utama
user_features = user_features.join(type_prefs)

print("Content Preference Features Complete.")
print(user_features.head())

Engineering Content Preference Features...
Content Preference Features Complete.
                         total_explicit_interactions  \
user_id                                                
30      completion_rate                           15   
        drop_off_rate                             15   
72      completion_rate                            8   
        drop_off_rate                              8   
114     completion_rate                           82   

                         total_implicit_interactions  avg_watch_pct  \
user_id                                                               
30      completion_rate                          221      21.333333   
        drop_off_rate                            221      21.333333   
72      completion_rate                           22      99.125000   
        drop_off_rate                             22      99.125000   
114     completion_rate                          168      98.426829   

                            

In [9]:
print("Engineering Topic Preference Features...")

# 1. Topic Specialization (Jumlah software unik yang ditonton)
# Kita harus memfilter 'No Tag' SEBELUM menghitung .nunique()
tagged_interactions = merged_df[merged_df['Software'] != 'No Tag']
user_features['topic_specialization'] = tagged_interactions.groupby('user_id')['Software'].nunique()

# 2. Preferensi Topik (Top 5 Software)
# Mari kita cari 5 software teratas (setelah memfilter 'No Tag')
top_5_software = tagged_interactions['Software'].value_counts().head(5).index.tolist()
print(f"Top 5 Software (excluding 'No Tag'): {top_5_software}")

# Buat fitur hanya untuk top 5 ini
software_prefs = tagged_interactions.groupby('user_id')['Software'].value_counts(normalize=True).unstack(fill_value=0)

# Hanya simpan kolom yang ada di top_5_software
for col in top_5_software:
    if col in software_prefs.columns:
        # Bersihkan nama kolom (misal: "['Excel']" -> "pref_sw_excel_pct")
        clean_col_name = f"pref_sw_{col.replace('[','').replace(']','').split(' ')[0].lower()}_pct"
        user_features[clean_col_name] = software_prefs[col]

print("Topic Preference Features Complete.")
print(user_features.head())

Engineering Topic Preference Features...
Top 5 Software (excluding 'No Tag'): ["['Microsoft Teams']", "['Outlook']", "['Excel']", "['OneDrive']", "['PowerPoint']"]
Topic Preference Features Complete.
                         total_explicit_interactions  \
user_id                                                
30      completion_rate                           15   
        drop_off_rate                             15   
72      completion_rate                            8   
        drop_off_rate                              8   
114     completion_rate                           82   

                         total_implicit_interactions  avg_watch_pct  \
user_id                                                               
30      completion_rate                          221      21.333333   
        drop_off_rate                            221      21.333333   
72      completion_rate                           22      99.125000   
        drop_off_rate                             22

## 6. Finalisasi & Penyimpanan

Langkah terakhir adalah membersihkan `NaN` (jika ada) dan menyimpan file CSV.

In [10]:
print("Finalizing feature table...")

# Periksa NaN. NaN bisa muncul jika user tidak punya interaksi (misal 'total_implicit_interactions')
# atau jika semua interaksinya adalah 'No Tag' (misal 'topic_specialization')
print(f"Total NaN values before fill: {user_features.isna().sum().sum()}")

# Mengisi semua NaN dengan 0. Ini logis, karena NaN berarti "tidak ada aktivitas" pada fitur tersebut.
user_features = user_features.fillna(0)

print(f"Total NaN values after fill: {user_features.isna().sum().sum()}")

# Simpan ke file CSV
output_filename = os.path.join(OUTPUT_DIR, 'user_features.csv')
user_features.to_csv(output_filename)

print(f"\nFeature Engineering Selesai! 🚀")
print(f"Tabel fitur disimpan di: {output_filename}")
print(f"Bentuk tabel fitur final: {user_features.shape}")
print(user_features.head())

Finalizing feature table...
Total NaN values before fill: 37688
Total NaN values after fill: 0

Feature Engineering Selesai! 🚀
Tabel fitur disimpan di: ../data/user_features.csv
Bentuk tabel fitur final: (5384, 14)
                         total_explicit_interactions  \
user_id                                                
30      completion_rate                           15   
        drop_off_rate                             15   
72      completion_rate                            8   
        drop_off_rate                              8   
114     completion_rate                           82   

                         total_implicit_interactions  avg_watch_pct  \
user_id                                                               
30      completion_rate                          221      21.333333   
        drop_off_rate                            221      21.333333   
72      completion_rate                           22      99.125000   
        drop_off_rate                